# Using connectivity from Snudda in NEST

This example shows how to create a network in Snudda, and then import the network into NEST to run the simulation there instead of in the default NEURON environment.

## Creating the network connectivity

In [ ]:
import os
network_path = os.path.join("networks","snudda_in_nest")

In [ ]:
from snudda import SnuddaInit

snudda_data = os.path.join("..", "..", "..", "..", "BasalGangliaData", "data")  # "/home/hjorth/HBP/BasalGangliaData/data/" 
snudda_data = "/home/hjorth/HBP/BasalGangliaData/data/" 
si = SnuddaInit(network_path=network_path, random_seed=12345, snudda_data=snudda_data)
si.define_striatum(num_dSPN=500, num_iSPN=500, num_FS=10, num_LTS=0, num_ChIN=0, neuron_density=80500,
                    volume_type="cube", neurons_dir="$DATA/neurons")
si.write_json()

In [ ]:
from snudda import SnuddaPlace
spl = SnuddaPlace(network_path=network_path)
spl.place()

In [ ]:
from snudda import SnuddaDetect

sd = SnuddaDetect(network_path=network_path)
sd.detect()

In [ ]:
from snudda import SnuddaPrune

sp = SnuddaPrune(network_path=network_path)
sp.prune()

# Setup input 

-- TODO: Check that NEST imports the input spikes

In [ ]:
input_config = {
    "dSPN": {
        "Ctx" : {
            "generator" : "poisson",
            "start" : [0, 4, 5],
            "end" : [4, 5, 10],
            "frequency" : [2, 4, 2],
            "conductance" : 0.5e-9,
            "nInputs" : 20,
            "populationUnitCorrelation" : [0.5, 1.0, 0.5],
            "jitter" : 0.01,	    
            "modFile": "tmGlut"
        }
    },

    "iSPN": {
        "Ctx" : {
            "generator" : "poisson",
            "start" : [0, 5],
            "end" : [3, 10],	    
            "frequency" : 1,
            "conductance" : 0.5e-9,
            "nInputs" : 10,
            "jitter" : 0.01,
            "populationUnitCorrelation" : 0.2,
            "populationUnitID" : 1,
            "modFile": "tmGlut"
        }
    },
    
    "FS": {
        "Ctx" : {
            "generator" : "poisson",
            "start" : [0, 5],
            "end" : [3, 10],	    
            "frequency" : 1,
            "conductance" : 0.5e-9,
            "nInputs" : 10,
            "jitter" : 0.01,
            "populationUnitCorrelation" : 0.2,
            "populationUnitID" : 1,
            "modFile": "tmGlut"
        }
    }
}

from snudda.input import SnuddaInput
si = SnuddaInput(network_path=network_path, 
                 input_config_file=input_config,
                 verbose=False)
si.generate()

In [ ]:
from snudda.utils.export_sonata import ExportSonata
se = ExportSonata(network_path=network_path)

In [ ]:
si = None
spl = None
sd = None
sp = None
se = None

# Import network into NEST and run simulation

In [ ]:
import nest

nest.ResetKernel()

base_dir = os.path.join(network_path, "SONATA")
net_config = os.path.join(base_dir, "circuit_config.json")
sim_config = os.path.join(base_dir,"simulation_config.json")

sonata_net = nest.SonataNetwork(net_config, sim_config)
# node_collections = sonata_net.Create()

node_collections = sonata_net.BuildNetwork()

s_rec_dspn = nest.Create("spike_recorder")
s_rec_ispn = nest.Create("spike_recorder")
s_rec_fs = nest.Create("spike_recorder")

# record_node_ids = [1, 80, 160, 240, 270]
# nest.Connect(node_collections[pop_name][record_node_ids], s_rec)

nest.Connect(node_collections["dSPN"], s_rec_dspn)
nest.Connect(node_collections["iSPN"], s_rec_ispn)
nest.Connect(node_collections["FS"], s_rec_fs)

In [ ]:
# If we need to add separate noise. 
# Currently the SONATA specified inputs exist (but according to SONATA documentation it should target virtual neurons), but are not properly connected.

exc_rate = 4.5

exc_noise_fs = nest.Create('poisson_generator', 1300)
exc_noise_dspn = nest.Create('poisson_generator', 1500)
exc_noise_ispn = nest.Create('poisson_generator', 1050)

exc_noise_fs.set(rate=exc_rate)
exc_noise_dspn.set(rate=exc_rate)
exc_noise_ispn.set(rate=exc_rate)

nest.Connect(exc_noise_fs, node_collections["FS"], 'all_to_all', {'weight': +0.5})
nest.Connect(exc_noise_dspn, node_collections["dSPN"], 'all_to_all', {'weight': +0.5})
nest.Connect(exc_noise_ispn, node_collections["iSPN"], 'all_to_all', {'weight': +0.5})

In [ ]:
import pandas as pd
conns = nest.GetConnections()
df = pd.DataFrame.from_dict(conns.get())

In [ ]:
df

In [ ]:
import numpy as np
np.sum(df["synapse_id"]==18)

In [ ]:
sonata_net.Simulate()

In [ ]:
import matplotlib.pyplot as plt
nest.raster_plot.from_device(s_rec_dspn)
nest.raster_plot.from_device(s_rec_ispn)
nest.raster_plot.from_device(s_rec_fs)
plt.show()

In [ ]:
os.getcwd()

In [ ]:
node_collections.keys()

In [ ]:
node_collections["Striatum-input"]

In [ ]:
node_collections["Striatum-input"]